<a href="https://colab.research.google.com/github/JoshuaWilbur/Dog_Emotion_Detector/blob/main/Dog_Emotion_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
IMAGE_DIR = "images"
FOLDERS = ["happy", "sad", "relaxed"]

DEVICE = "cuda"

In [ ]:
import os
import pandas as pd
import torch
import numpy as np
import torchvision.transforms as transforms

In [19]:
img_files = []
labels = []
label_code = []

for folder in FOLDERS:
    fname = os.path.join("/content/drive/MyDrive/images", folder)

    for im in os.listdir(fname):
        impath= os.path.join(fname, im)

        img_files.append(impath)
        labels.append(folder)
        label_code.append(FOLDERS.index(folder))

In [20]:
dataset = pd.DataFrame(dict(filename = img_files, label = labels, code = label_code))

In [21]:
dataset

,filename,label,code
0,/content/drive/MyDrive/images/happy/7182058879...,happy,0
1,/content/drive/MyDrive/images/happy/4991302155...,happy,0
2,/content/drive/MyDrive/images/happy/6869292594...,happy,0
3,/content/drive/MyDrive/images/happy/3419969897...,happy,0
4,/content/drive/MyDrive/images/happy/5918050080...,happy,0
...,...,...,...
13660,/content/drive/MyDrive/images/relaxed/25268356...,relaxed,2
13661,/content/drive/MyDrive/images/relaxed/64004153...,relaxed,2
13662,/content/drive/MyDrive/images/relaxed/50855491...,relaxed,2
13663,/content/drive/MyDrive/images/relaxed/21214338...,relaxed,2


In [22]:
import torch

In [23]:
torch.manual_seed(0)

In [24]:
device = torch.device(DEVICE)

In [25]:
from torchvision.io import read_image
from torchvision import transforms as T
from torch.utils.data import Dataset
import math

class DogDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.augments = [T.RandomHorizontalFlip(1), T.RandomRotation(90), T.AutoAugment()]
        self.normalize = T.Compose([
            T.ConvertImageDtype(torch.float),
            T.Normalize((.485, .456, 406), (.229, .224, .225))
        ])

    def __len__(self):
        return self.dataset.shape[0] * (len(self.augments) + 1)

    def classes(self):
        return self.dataset["code"].unique()

    def __getitem__(self, idx):
        augment = math.floor(idx / self.dataset.shape[0])
        idx = idx % self.dataset.shape[0]

        row = self.dataset.iloc[idx,:]

        img_path = row["filename"]
        image = read_image(img_path)

        label = row["code"]

        if augment > 0:
            image = self.augments[augment - 1].forward(image)

        image = self.normalize(image)

        return image, int(label), img_path

data = DogDataset(dataset)

In [26]:
train_size = int(0.8 * len(data))
test_size = len(data) - train_size

train_data, test_data = torch.utils.data.random_split(data, [train_size, test_size], generator = torch.Generator().manual_seed(1))

In [27]:
from torch.utils.data import DataLoader

BATCH_SIZE = 64

EPOCHS = 50

train = DataLoader(train_data, batch_size = BATCH_SIZE, shuffle = True)
test = DataLoader(test_data, batch_size = BATCH_SIZE, shuffle = True)

In [28]:
from torch import nn

class NeuralNetwork(nn.Module):
    def __init__(self, classes):
        super(NeuralNetwork, self).__init__()

        self.bn = nn.BatchNorm2d(64)

        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 4, stride = 2),
            self.bn,
            nn.ReLU(True),
            nn.MaxPool2d(2,2),
            nn.Conv2d(64,64,2),
            self.bn,
            nn.Conv2d(64, 64, 2),
            self.bn,
            nn.ReLU(True),
            nn.MaxPool2d(2,2)
        )

        self.dense = nn.Sequential(
            nn.Linear(64 * 46 * 46, 64),
            nn.Linear(64, len(classes))
        )

    def forward(self, x):
        x = self.cnn(x)
        x = torch.flatten(x, 1)
        x=self.dense(x)
        return x

In [29]:
labels = data.classes()

In [30]:
labels

array([0, 1, 2])

In [31]:
model = NeuralNetwork(labels).to(device)


In [32]:
model

NeuralNetwork(
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (cnn): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 64, kernel_size=(2, 2), stride=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Conv2d(64, 64, kernel_size=(2, 2), stride=(1, 1))
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (dense): Sequential(
    (0): Linear(in_features=135424, out_features=64, bias=True)
    (1): Linear(in_features=64, out_features=3, bias=True)
  )
)

In [33]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = .001)

In [34]:
size = len(train.dataset)

In [35]:
for epoch in range(EPOCHS):
    for batch, (images, labels, img_paths) in enumerate(train):
        optimizer.zero_grad()

        images = images.to(device)
        pred = model(images.float())
        labels = labels.to(device)
        loss = loss_fn(pred, labels)

        loss.backward()
        optimizer.step()

    loss, current = loss.item()
    print(f"loss: {loss:>7f} [{epoch}]")

TypeError: ignored